In [1]:
import jmespath
import requests
import json
import pandas as pd
import numpy as np

In [2]:
jFile = open("data.json")
data = json.load(jFile)

#2. How many cities  (don't include subregions) where Swiggy is having their restaurants listed?
print(len(data.keys()))

# print(data.keys())

623


In [3]:
count=0
for key in data.keys():
    count+=len(data[key])-1
#     print(len(data[key]))
# 1. How many cities (including subregions) where Swiggy is having its restaurants listed?

print(count)

972


In [4]:
# 3. The Subregion of Delhi with the maximum number of restaurants listed on Swiggy?
max = 0
maxSr = ""
for key in data['Delhi'].keys():
    if max < len(data['Delhi'][key]['restaurants']):
                 max = len(data['Delhi'][key]['restaurants'])
                 maxSr = key
print(maxSr, max)
    

Indirapuram 1279


In [5]:
Cities = list(data.keys())
# Cities

In [6]:
Data = pd.DataFrame()                               # Create an Empty Dataframe

for city in Cities:
    try:
        # Convert the city-wise restaurant data to pandas Dataframe
        DF = pd.DataFrame(data[city]['restaurants']).T  
        
        # add City column for each restaurants in that city
        City = [city for i in range(DF.shape[0])]
        DF['City'] = City
        
        # Because no sub-region so considering city as sub-region
        SubRegion = City                                          
        DF['SubRegion'] = SubRegion
        
        # Concatenate to main Dataframe
        Data = pd.concat([Data,DF])
        
    except:
        
        Subregions = list(data[city].keys())
        for region in Subregions:
            
            # Convert the city-wise restaurant data to pandas Dataframe
            DF = pd.DataFrame(data[city][region]['restaurants']).T
            
            # City
            City = [city for i in range(DF.shape[0])]
            # Add City column for each restaurants in that city
            DF['City'] = City
            
            # Sub-region
            SubRegion = [region for i in range(DF.shape[0])]
            
            # add sub-region column for each restaurants in that city
            DF['SubRegion'] = SubRegion
            Data = pd.concat([Data,DF])
        
Data.shape

(181404, 13)

In [7]:
Data.to_csv("Swiggy.csv")

In [8]:
df = pd.read_csv("Swiggy.csv")

In [9]:
menus = []
for i in range(Data.shape[0]):
    try:
        menus.append(', '.join([key for key in Data.menu[i].keys()]))
    except:
        menus.append(np.NaN)
print(len(menus))
print(df.shape[0])

df['menu'] = menus

181404
181404


In [10]:
df = df.rename(columns={'Unnamed: 0':'id', 'name':'Restaurant'})
df.drop_duplicates(subset='id', keep='first', inplace = True)
#df.set_index('id', inplace=True)
df.shape
df.columns

Index(['id', 'Restaurant', 'rating', 'rating_count', 'cost', 'address',
       'cuisine', 'lic_no', 'menu', 'link', 'City', 'SubRegion', 'type',
       'price'],
      dtype='object')

In [11]:
df['cost'] = df['cost'].str.replace('₹', '').astype(float)

In [12]:
# 4. Name the top 5 Most Expensive Cities in the Datasets.
top_expensive_cities = df.groupby('City')['cost'].mean().sort_values(ascending=False).head(5)

print(top_expensive_cities)

City
Hinganghat    18962.437500
North-goa       468.964045
Gangtok         466.666667
Mussoorie       460.000000
South-goa       442.857143
Name: cost, dtype: float64


In [13]:
# 5. List out the top 5 Restaurants with Maximum & minimum ratings throughout the dataset.

df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Get top 5 restaurants with maximum ratings
top_rated_restaurants = df.nlargest(5, 'rating')

# Get top 5 restaurants with minimum ratings
bottom_rated_restaurants = df.nsmallest(5, 'rating')

print("Top 5 Restaurants with Maximum Ratings:")
print(top_rated_restaurants[['Restaurant', 'rating']])

print("\nTop 5 Restaurants with Minimum Ratings:")
print(bottom_rated_restaurants[['Restaurant', 'rating']])

Top 5 Restaurants with Maximum Ratings:
                            Restaurant  rating
757              Pablo Premium Kitchen     5.0
1022              The best diet corner     5.0
10962                 Creams and Bites     5.0
11012     Bansal's Thali & Combo Meals     5.0
11444  GOWHEY- HEALTHY DESSERT COMPANY     5.0

Top 5 Restaurants with Minimum Ratings:
                              Restaurant  rating
36944            Ice Cream and Shakes Co     1.0
19752                    Persian Delight     1.1
40027  SHAWARMA WRAP - ROLL YOUR SECRETS     1.2
57710                     THE TARI STORY     1.2
75201       The Hyderabadi Biryani House     1.2


In [14]:
# 6. Name of top 5 cities with the highest number of restaurants listed.

top_cities_restaurants = df['City'].value_counts().head(5)

print("Top 5 Cities with the Highest Number of Restaurants:")
print(top_cities_restaurants)

Top 5 Cities with the Highest Number of Restaurants:
Bangalore    15358
Delhi        14147
Pune         12472
Hyderabad    11912
Chennai      10089
Name: City, dtype: int64


In [15]:
# 7. Top 10 cities as per the number of restaurants listed?
top_cities_restaurants = df['City'].value_counts().head(10)

print("Top 10 Cities with the Highest Number of Restaurants:")
print(top_cities_restaurants)

Top 10 Cities with the Highest Number of Restaurants:
Bangalore     15358
Delhi         14147
Pune          12472
Hyderabad     11912
Chennai       10089
Kolkata        8418
Mumbai         6734
Jaipur         5567
Ahmedabad      3809
Chandigarh     3348
Name: City, dtype: int64


In [16]:
# 8. Name the top 5 Most Popular Restaurants in Pune.
pune_restaurants = df[df['City'] == 'Pune']  # Filter restaurants in Pune
pune_popular_restaurants = pune_restaurants.sort_values(by=['rating', 'rating_count'], ascending=[False, False]).head(5)

print("Top 5 Most Popular Restaurants in Pune:")
print(pune_popular_restaurants[['Restaurant', 'rating', 'rating_count']])

Top 5 Most Popular Restaurants in Pune:
                                Restaurant  rating rating_count
146602                      The Samosa Man     5.0  20+ ratings
148082                       HRX by Eatfit     5.0  20+ ratings
153561                     CIRCLE OF CRUST     5.0  20+ ratings
154116  Smoodies & Juices: Your Fruitfully     5.0  20+ ratings
154277                       HRX by Eatfit     5.0  20+ ratings


In [17]:
# 9. Which SubRegion in Delhi is having the least expensive restaurant in terms of cost?
delhi_restaurants = df[df['City'] == 'Delhi']  # Filter restaurants in Delhi
cheapest_delhi_subregion = delhi_restaurants.sort_values(by='cost').iloc[0]['SubRegion']

print("SubRegion in Delhi with the Least Expensive Restaurant:")
print(cheapest_delhi_subregion)

SubRegion in Delhi with the Least Expensive Restaurant:
Dilshad Gardens


In [18]:
# 10. Top 5 most popular restaurant chains in India?
restaurant_chains = df.groupby('Restaurant').agg({'rating': 'mean', 'rating_count': 'sum'}).reset_index()
top_popular_chains = restaurant_chains.sort_values(by=['rating', 'rating_count'], ascending=[False, False]).head(5)

print("Top 5 Most Popular Restaurant Chains in India:")
print(top_popular_chains[['Restaurant', 'rating', 'rating_count']])

Top 5 Most Popular Restaurant Chains in India:
                                        Restaurant  rating  \
29716                 Dessert Studio by Third Wave     5.0   
85894                                Rollie Rollie     5.0   
50114  JUST CREAMERY - Artisanal Healthy Ice Cream     5.0   
64485                              Mamaji PavBhaji     5.0   
7353                                     Apna food     5.0   

                                            rating_count  
29716  Too Few RatingsToo Few RatingsToo Few RatingsT...  
85894  Too Few RatingsToo Few RatingsToo Few Ratings2...  
50114   Too Few Ratings20+ ratings20+ ratings20+ ratings  
64485              Too Few Ratings20+ ratings20+ ratings  
7353                          Too Few Ratings20+ ratings  


In [19]:
# 11. Which restaurant in Pune has the most number of people visiting?
pune_restaurants = df[df['City'] == 'Pune']  # Filter restaurants in Pune
most_visited_pune_restaurant = pune_restaurants.sort_values(by='rating_count', ascending=False).iloc[0]['Restaurant']

print("Restaurant in Pune with the Most Number of People Visiting:")
print(most_visited_pune_restaurant)

Restaurant in Pune with the Most Number of People Visiting:
FARM DA DHABA NH8


In [20]:
# 12. Top 10 Restaurants with Maximum Ratings in Banglore
bangalore_restaurants = df[df['City'] == 'Bangalore']  # Filter restaurants in Bangalore
top_rated_bangalore_restaurants = bangalore_restaurants.sort_values(by='rating', ascending=False).head(10)

print("Top 10 Restaurants with Maximum Ratings in Bangalore:")
print(top_rated_bangalore_restaurants[['Restaurant', 'rating']])

Top 10 Restaurants with Maximum Ratings in Bangalore:
                         Restaurant  rating
25119  Provenance Deli and Desserts     5.0
14452                      Minus 30     5.0
20659            Shahenshah Biryani     5.0
20683                       Bowl 99     5.0
22375                Bombay 2 Delhi     5.0
20091                 HRX by Eatfit     5.0
13350                 HUNGER TREATS     5.0
13374                    Snack Ride     5.0
14265            Fresh crunch pizza     5.0
17807                  PEDRO DE GOA     5.0


In [21]:
# 13. Top 10 Restaurant in Patna w.r.t rating 
patna_restaurants = df[df['City'] == 'Patna']  # Filter restaurants in Patna
top_rated_patna_restaurants = patna_restaurants.sort_values(by='rating', ascending=False).head(10)

print("Top 10 Restaurants in Patna by Rating:")
print(top_rated_patna_restaurants[['Restaurant', 'rating']])

Top 10 Restaurants in Patna by Rating:
                                             Restaurant  rating
145066  Kwality Walls Frozen Dessert and Ice Cream Shop     4.6
145266                                    Slurpy Shakes     4.6
145300                    Cupcake Bliss Cake & Desserts     4.5
145323                                     Burger It Up     4.5
145254                      Kouzina Kafe The Food Court     4.5
145555                                   GARAGE KITCHEN     4.5
145283                                 The Dessert Zone     4.4
145332                       Pure Veg Meals by Lunchbox     4.4
145509                                 KINGDOM OF MOMOS     4.4
145272                                  Indiana Burgers     4.4
